# SPARQL: Die RDF Abfragesprache

In dieser Übung schauen wir uns die SPARQL Abfragesprache für RDF in der Praxis an. Wir verwenden dafür etwas RDF mit der Pink Floyd Diskographie und führen einige Abfragen aus. Zum Schluss schreiben Sie dann Ihr eigenes RDF und einige SPARQL Abfragen dazu.

In [2]:
!pip install rdflib

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
from io import BytesIO, StringIO
from rdflib import Graph
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer
from IPython.display import display

rdf = """
@prefix ex: <http://example.org#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a ex:Album ;
   ex:title "The Dark Side of the Moon"^^xsd:string ;
   ex:label "Harvest, EMI"@en ;
   ex:released [ 
     ex:day "16"^^xsd:int ;
     ex:month "03"^^xsd:int ;
     ex:year "1973"^^xsd:int 
   ] .
   
[] a ex:Album ;
   ex:title "The Wall" ;
   ex:label "Harvest, EMI" ;
   ex:released [ 
     ex:day 30 ;
     ex:month "11"^^xsd:string ;
     ex:year "1979"^^xsd:int 
   ] .

[] a ex:Single ;
   ex:title "What God Wants, Part 1"^^xsd:string ;
   ex:author [
     ex:firstname "Roger" ;
     ex:lastname "Waters"
   ] ;
   ex:released [ 
     ex:year "1992"^^xsd:int 
   ] .
"""

g = Graph()

r = g.parse(data=rdf, format='turtle')

def query(q):
    serializer = CSVResultSerializer(g.query(q))
    output = BytesIO()
    serializer.serialize(output)
    display(pd.read_csv(StringIO(output.getvalue().decode())))

Führen Sie nun die folgenden Abfragen aus und beantworten Sie die Fragen.

In [4]:
query("""
SELECT ?title WHERE { 
  ?work ex:title ?title
}
""")
# gibt alle Objekte mit dem Prädikt title aus

,title
0,"What God Wants, Part 1"
1,The Wall
2,The Dark Side of the Moon


In [5]:
query("""
SELECT ?work ?title WHERE { 
  ?work ex:title ?title
}
""")
# gibt alle Subjekte und Objekte aus bei denen das Prädikat title ist

,work,title
0,ub2bL23C1,"What God Wants, Part 1"
1,ub2bL14C1,The Wall
2,ub2bL5C1,The Dark Side of the Moon


In [6]:
query("""
SELECT ?title WHERE { 
  [] ex:title ?title
}
""")

# Was ist der Unterschied zum vorherigem Beispiel? Antwort: hier wird das Objekt angezeigt wenn das Prädikat title ist und
# das Subject ein Blank node

,title
0,"What God Wants, Part 1"
1,The Wall
2,The Dark Side of the Moon


In [7]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ; 
     ex:title ?title
}
""")

# Warum nur zwei Resultate? Antwort: weil die Where-Bedingung "  [] rdf:type ex:Album " nur bei zwei einträgen stimmt und beide
# Bedingungen erfüllt werden müssen

,title
0,The Dark Side of the Moon
1,The Wall


In [8]:
query("""
SELECT ?s ?p ?o WHERE { 
  ?s ?p ?o
}
""")

# Was erhält man hier? Antwort: Alle angegeben triple

,s,p,o
0,ub2bL8C16,http://example.org#year,1973
1,ub2bL25C14,http://example.org#firstname,Roger
2,ub2bL5C1,http://example.org#released,ub2bL8C16
3,ub2bL14C1,http://example.org#title,The Wall
4,ub2bL5C1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org#Album
5,ub2bL23C1,http://example.org#author,ub2bL25C14
6,ub2bL8C16,http://example.org#day,16
7,ub2bL23C1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org#Single
8,ub2bL25C14,http://example.org#lastname,Waters
9,ub2bL14C1,http://example.org#label,"Harvest, EMI"


In [9]:
query("""
SELECT ?title ?year WHERE { 
  [] rdf:type ex:Album ; 
     ex:title ?title ;
     ex:released [ ex:year ?year ]
  FILTER (?year > 1973)
}
""")
# sehr präzise Abfrage nach allen Alben die nach 1973 veröffentlicht wurden und
# bei denen die oben angegeben tripel vorhanden sind

,title,year
0,The Wall,1979


In [10]:
query("""
SELECT ?title ?year WHERE { 
  {
    [] rdf:type ex:Album ; 
       ex:title ?title ;
       ex:released [ ex:year ?year ]
    FILTER (?year > 1973)
  }
  UNION
  {
    [] rdf:type ex:Single ; 
       ex:title ?title ;
       ex:released [ ex:year ?year ]
    FILTER (?year <= 2000 )
  }
}
""")
# ähnlich wie eben, jetzt aber mit einer zusätzlichen Where-Bedingung

,title,year
0,The Wall,1979
1,"What God Wants, Part 1",1992


In [11]:
query("""
SELECT ?title ?label WHERE { 
    ?work ex:title ?title .
    OPTIONAL { ?work ex:label ?label }
}
""")

# Warum ist der label von "What God Wants, Part 1" NaN? 
# Antwort: like null... Not a Number... er findet kein Label und sagt da ist kein Eintrag

,title,label
0,"What God Wants, Part 1",NaN
1,The Wall,"Harvest, EMI"
2,The Dark Side of the Moon,"Harvest, EMI"


In [12]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ;
     ex:title ?title ;
     ex:label ?label
  FILTER (LANG(?label) = "en")
}
""")
# gibt alle triple bei denen der Aufbau wie oben 
# gegeben ist und die Sprache Englisch ist

,title
0,The Dark Side of the Moon


In [13]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ;
     ex:title ?title ;
     ex:released [ ex:day ?day ]
  FILTER (?day > 15)
}
""")

,title
0,The Dark Side of the Moon
1,The Wall


In [ ]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ;
     ex:title ?title ;
     ex:released [ ex:month ?month ]
  FILTER (DATATYPE(?month) = xsd:string)
}
""")

# Warum erhält man hier nur "The Wall" als Resultat? 
# Antwort: Wegen dem Filter String... der andere ist ein int

Schreiben Sie nun ihr eigenes RDF und werten Sie einige SPARQL Abfragen aus.

In [ ]:
rdf = """
... Mein RDF ...
"""

g = Graph()

r = g.parse(data=rdf, format='turtle')

In [ ]:
query("""
... Meine SPARQL Abfrage ...
""")